In [1]:
import foolbox
import keras
import tensorflow as tf
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.layers import Conv2D, Activation, Flatten, Dense
from keras.models import Sequential
from keras.optimizers import SGD

from cleverhans.utils_mnist import data_mnist

import time

Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Generating a model
sess = tf.Session()
X_train, Y_train, X_test, Y_test = data_mnist(train_start=0,
                                              train_end=60000,
                                              test_start=0,
                                              test_end=10000)

Extracting /tmp/train-images-idx3-ubyte.gz
Extracting /tmp/train-labels-idx1-ubyte.gz
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz
X_train shape: (60000, 28, 28, 1)
X_test shape: (10000, 28, 28, 1)


In [3]:
model = Sequential()
layers = [Conv2D(64, (8, 8), strides=(2, 2), padding="same",
                      input_shape=(28,28,1)),
          Activation('relu'),
          Conv2D((64 * 2), (6, 6), strides=(2, 2), padding="valid"),
          Activation('relu'),
          Conv2D((64 * 2), (5, 5), strides=(1, 1), padding="valid"),
          Activation('relu'),
          Flatten(),
          Dense(10)]
for layer in layers:
    model.add(layer)
model.add(Activation('softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [4]:
model.fit(X_train, Y_train, epochs=6, batch_size=128, validation_data = (X_test, Y_test))
model.layers.pop()
flmodel = foolbox.models.KerasModel(model, bounds=(0,1), predicts="logits")

Train on 60000 samples, validate on 10000 samples
Epoch 1/6
60000/60000 [==============================] - 7s 125us/step - loss: 0.3808 - acc: 0.8850 - val_loss: 0.1168 - val_acc: 0.9644
Epoch 2/6
60000/60000 [==============================] - 6s 106us/step - loss: 0.0997 - acc: 0.9703 - val_loss: 0.0749 - val_acc: 0.9762
Epoch 3/6
60000/60000 [==============================] - 6s 106us/step - loss: 0.0686 - acc: 0.9785 - val_loss: 0.0612 - val_acc: 0.9797
Epoch 4/6
60000/60000 [==============================] - 6s 106us/step - loss: 0.0531 - acc: 0.9841 - val_loss: 0.0611 - val_acc: 0.9803
Epoch 5/6
60000/60000 [==============================] - 6s 106us/step - loss: 0.0427 - acc: 0.9866 - val_loss: 0.0500 - val_acc: 0.9833
Epoch 6/6
60000/60000 [==============================] - 6s 106us/step - loss: 0.0354 - acc: 0.9891 - val_loss: 0.0402 - val_acc: 0.9872


In [5]:
from foolbox.criteria import Misclassification
from foolbox.distances import MeanSquaredDistance
from foolbox.attacks import FGSM, DeepFoolAttack, ApproximateLBFGSAttack, BoundaryAttack, SinglePixelAttack

import sys
import io
untar_crit = Misclassification()
atk_bound = BoundaryAttack(flmodel, untar_crit)
save_stdout = sys.stdout

In [ ]:

sys.stdout = save_stdout
# Experiments for 1000 Iters
bound_dist = [np.zeros(100) - 1 for i in range(5)]
bound_num_correct = [0 for i in range(5)]
bound_time = [0 for i in range(5)]
for max_iter in [1000,2000,3000,4000,5000]:
    start = time.time()
    slot = int(max_iter/1000 - 1)
    print(slot)
    for i in range(50):
        if (i % 5 == 0):
            print(str(i*2) + "% complete")
        inp,lab=X_test[i],Y_test[i]
        lab = np.argmax(lab)
        
        sys.stdout = io.StringIO()
        adv = atk_bound(inp,lab,iterations=max_iter)
        sys.stdout = save_stdout

        if adv is None:
            bound_num_correct[slot] += 1
        elif (np.argmax(model.predict(np.array([adv]))) != lab):
            bound_dist[slot][i]=MeanSquaredDistance(inp,adv,(0,1)).value
        else:
            bound_num_correct[slot] += 1
    a = time.time() - start
    print(a)
    print(np.median(bound_dist[slot][bound_dist[slot] >= 0]))
    bound_time[slot] = a

0
0% complete
10% complete
20% complete
30% complete
40% complete
50% complete
60% complete
70% complete
80% complete
90% complete
724.2450070381165
0.00326445288196
1
0% complete
10% complete
20% complete
30% complete
40% complete
50% complete
60% complete
70% complete
80% complete
90% complete
1203.1902935504913
0.00282748073948
2
0% complete
10% complete
20% complete
30% complete
40% complete
50% complete
60% complete
70% complete
80% complete
90% complete
1694.1352043151855
0.002547927809
3
0% complete
10% complete
20% complete
30% complete
40% complete
50% complete
60% complete


In [8]:
bound_time

[285.5978436470032,
 476.7730576992035,
 668.3349094390869,
 833.452091217041,
 849.0178511142731]